## Data clean-up

In [17]:
import pandas as pd

In [18]:
# import csv data to pandas dataframe
# row 0 includes DSpace property names (ignore)
fh = '../files/coll151_metadata_17-10-29.csv'
orig_df = pd.read_csv(fh, header=1, skiprows=0)

In [19]:
# list columns
orig_df.columns

Index([u'Collection', u'Collection_Description', u'File_Name', u'Identifier',
       u'Title', u'Personal_Name', u'Corporate_Name', u'Role',
       u'Subject_Topic', u'Subject_Personal_Name', u'Subject_Corporate_Name',
       u'Location', u'Time_Period', u'Date_Created', u'Date_Issued',
       u'Publisher', u'Place_of_Publication', u'Description', u'Note', u'Type',
       u'Genre', u'Form', u'Extent', u'Language', u'Related_Resource',
       u'Rights', u'Owning_Institution', u'Digital_Format', u'Digital_Origin'],
      dtype='object')

In [20]:
# show first 5 rows in dataframe
orig_df.head(5)

,Collection,Collection_Description,File_Name,Identifier,Title,Personal_Name,Corporate_Name,Role,Subject_Topic,Subject_Personal_Name,...,Type,Genre,Form,Extent,Language,Related_Resource,Rights,Owning_Institution,Digital_Format,Digital_Origin
0,World War II in Pictures (temporary title),The collection consist of photographs related ...,701-151-005-3057-005.tif,701-151-005-3057-005,Welcoming of general Sikorski in the Polish Ar...,NaN,NaN,Author,World War (1939-1945),"Okulicki, Leopold;Sikorski, Władysław;Anders, ...",...,still image,photographs,black-and-white photographs,19x25 cm,pl,NaN,TBD,Józef Pilsudski Institute of America,image/tiff,digitized other analog
1,World War II in Pictures (temporary title),The collection consist of photographs related ...,701-151-005-3057-010.tif,701-151-005-3057-010,"General Władysław Sikorski, Polish Premier, ch...",NaN,NaN,Author,World War II (1939-1945),"Sikorski, Władysław;Tokarzewski-Karaszewicz, M...",...,still image,photographs,black-and-white photographs,12x17 cm,pl,NaN,TBD,Józef Pilsudski Institute of America,image/tiff,digitized other analog
2,World War II in Pictures (temporary title),The collection consist of photographs related ...,701-151-005-3057-014.tif,701-151-005-3057-014,"General Sikorski in Russia, talking to the pol...",NaN,NaN,Author,World War (1939-1945),"Sikorski, Władysław",...,still image,photographs,black-and-white photographs,14x19 cm,NaN,NaN,TBD,Józef Pilsudski Institute of America,image/tiff,digitized other analog
3,World War II in Pictures (temporary title),The collection consist of photographs related ...,701-151-005-3061-001.tif,701-151-005-3061-001,The military command of Polish Army in the Sov...,NaN,NaN,Author,World War (1939-1945),"Anders, Władysław;Okulicki, Leopold",...,still image,photographs,black-and-white photographs,13x18 cm,pl,NaN,TBD,Józef Pilsudski Institute of America,image/tiff,digitized other analog
4,World War II in Pictures (temporary title),The collection consist of photographs related ...,701-151-005-4949-001.tif,701-151-005-4949-001,Polish children before transport to India,NaN,NaN,Author,World War II (1939-1945);Children and war,NaN,...,still image,photographs,black-and-white photographs,12x12 cm,pl,NaN,TBD,Józef Pilsudski Institute of America,image/tiff,digitized other analog


In [21]:
# number of rows and columns in dataframe
orig_df.shape

(512, 29)

In [22]:
# since Collection & Collection_Description consist of the same values for all data,
# these elements can be omitted from the analysis
orig_df.Collection.unique()

array(['World War II in Pictures (temporary title)'], dtype=object)

In [23]:
orig_df.Collection_Description.unique()

array([ 'The collection consist of photographs related to Polish Armed Forces in the West, Polish Army in Russia and their journey to Italy \xe2\x80\x93  until they reached Monte Cassino, as well as fate of Polish civilians including children during the war and images of Polish Government- in-Exile officials. (to be refined)'], dtype=object)

### Verifying files and ids

* find missing data

In [24]:
# test row to be removed!
orig_df[orig_df['File_Name'].isnull()]

,Collection,Collection_Description,File_Name,Identifier,Title,Personal_Name,Corporate_Name,Role,Subject_Topic,Subject_Personal_Name,...,Type,Genre,Form,Extent,Language,Related_Resource,Rights,Owning_Institution,Digital_Format,Digital_Origin
5,World War II in Pictures (temporary title),The collection consist of photographs related ...,NaN,#VALUE!,Test Title in English,"Smith, John;Kowalski, Antoni",Author-company,Author,World War (1939-1945),"Sosnkowski, Kazimierz;Abamowicz, Konstanty",...,still image,photographs,black-and-white photographs,12x33 cm,NaN,NaN,TBD,Józef Pilsudski Institute of America,image/tiff,digitized other analog


In [25]:
# remove test row from dataframe
orig_df = orig_df.drop(orig_df.index[5])

In [26]:
orig_df[orig_df['Identifier'].isnull()]

,Collection,Collection_Description,File_Name,Identifier,Title,Personal_Name,Corporate_Name,Role,Subject_Topic,Subject_Personal_Name,...,Type,Genre,Form,Extent,Language,Related_Resource,Rights,Owning_Institution,Digital_Format,Digital_Origin


* find if files names are unique in each record

In [27]:
# create df for only file names and id
file_df = orig_df[['File_Name', 'Identifier']].copy()
s = file_df['File_Name'].str.split(';').apply(pd.Series, 1).stack()

In [28]:
s.index = s.index.droplevel(-1)  # to line up with file_df index

In [29]:
s.name = 'File_Name'

In [30]:
del file_df['File_Name']

In [31]:
file_df = file_df.join(s)

In [32]:
file_df.head(15)

,Identifier,File_Name
0,701-151-005-3057-005,701-151-005-3057-005.tif
1,701-151-005-3057-010,701-151-005-3057-010.tif
2,701-151-005-3057-014,701-151-005-3057-014.tif
3,701-151-005-3061-001,701-151-005-3061-001.tif
4,701-151-005-4949-001,701-151-005-4949-001.tif
6,701-151-005-2464-001,701-151-005-2464-001.tif
7,701-151-005-2505-001,701-151-005-2505-001.tif
8,701-151-005-2484-001,701-151-005-2484-001.tif
9,701-151-005-2487-001,701-151-005-2487-001.tif
9,701-151-005-2487-001,701-151-005-2487-002.tif


In [40]:
dups = file_df[file_df.duplicated(keep=False)].sort_values(['Identifier', 'File_Name'])

In [41]:
dups

,Identifier,File_Name
20,701-151-005-2534-001,701-151-005-2534-001.tif
406,701-151-005-2534-001,701-151-005-2534-001.tif
14,701-151-005-2573-001,701-151-005-2573-001.tif
33,701-151-005-2573-001,701-151-005-2573-001.tif
174,701-151-005-2673-001,701-151-005-2673-001.tif
259,701-151-005-2673-001,701-151-005-2673-001.tif
174,701-151-005-2673-001,701-151-005-2673-002.tif
259,701-151-005-2673-001,701-151-005-2673-002.tif
0,701-151-005-3057-005,701-151-005-3057-005.tif
501,701-151-005-3057-005,701-151-005-3057-005.tif
